# GI Hybrid Model Optimization with Optuna TPE + CatBoost

This notebook optimizes your trained hybrid model using:
- **Optuna TPE**: Hyperparameter optimization
- **CatBoost**: Meta-learner on top of hybrid features

## Expected Improvement: 89.58% → 91-93%

## Setup:
1. Enable GPU (Runtime → Change runtime type → T4 GPU)
2. Upload your trained `hybrid_model_best.pth`
3. Run all cells

## 1. Check GPU

In [ ]:
!nvidia-smi

import torch
print(f"\nPyTorch GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Install Packages

In [ ]:
!pip install -q tensorflow>=2.16.0
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q optuna catboost scikit-learn matplotlib seaborn pandas numpy

print("✅ All packages installed!")

## 3. Download Dataset

In [ ]:
import os
import zipfile
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

if not os.path.exists("kvasir-dataset-v2"):
    print("Downloading Kvasir dataset...")
    !wget --no-check-certificate -q https://datasets.simula.no/downloads/kvasir/kvasir-dataset-v2.zip
    
    print("Extracting...")
    with zipfile.ZipFile("kvasir-dataset-v2.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("kvasir-dataset-v2.zip")
    print("✅ Dataset ready!")
else:
    print("✅ Dataset already exists!")

data_dir = "kvasir-dataset-v2/"
classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
print(f"\nFound {len(classes)} classes: {classes}")

## 4. Upload Models from Google Drive

In [ ]:
from google.colab import drive
import shutil

drive.mount('/content/drive')

DRIVE_FOLDER = "/content/drive/MyDrive/GI_hybrid"

os.makedirs("mobvit", exist_ok=True)
os.makedirs("model_checkpoints", exist_ok=True)

# Copy files
files_to_copy = [
    (f"{DRIVE_FOLDER}/model.py", "mobvit/model.py"),
    (f"{DRIVE_FOLDER}/efficientnet_v2m_best_model.weights.h5", "model_checkpoints/efficientnet_v2m_best_model.weights.h5"),
    (f"{DRIVE_FOLDER}/hybrid_model_best.pth", "hybrid_model_best.pth"),
]

for src, dst in files_to_copy:
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f"✅ Copied {os.path.basename(dst)}")
    else:
        print(f"⚠️ {os.path.basename(src)} not found")

print("\n✅ Files ready!")

## 5. Load Models

In [ ]:
import sys
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

sys.path.append('./mobvit')
from model import MobileViT

# Create MobileViT
def create_mobilevit(num_classes=8):
    return MobileViT(
        image_size=224,
        num_classes=num_classes,
        dims=[96, 192, 384],
        channels=[16, 32, 64, 128, 160, 192, 256, 320, 384, 512],
        depths=[2, 4, 3]
    )

# Create EfficientNet
def create_efficientnet(num_classes=8):
    base_model = tf.keras.applications.EfficientNetV2M(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = True
    
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = tf.keras.applications.efficientnet_v2.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    
    return tf.keras.Model(inputs, outputs)

# Hybrid Model
class HybridGIModel(nn.Module):
    def __init__(self, num_classes=8):
        super(HybridGIModel, self).__init__()
        self.mobilevit = create_mobilevit(num_classes=num_classes)
        self.fusion_weights = nn.Parameter(torch.tensor([0.5, 0.5]))
        self.fusion_fc = nn.Sequential(
            nn.Linear(num_classes * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        self.num_classes = num_classes
        self.use_advanced_fusion = True
    
    def forward(self, x, efficientnet_logits=None, return_features=False):
        mobilevit_logits = self.mobilevit(x)
        
        if efficientnet_logits is None:
            return mobilevit_logits
        
        weights = F.softmax(self.fusion_weights, dim=0)
        combined = torch.cat([mobilevit_logits, efficientnet_logits], dim=1)
        
        if return_features:
            return combined  # Return concatenated features for CatBoost
        
        if self.use_advanced_fusion:
            output = self.fusion_fc(combined)
        else:
            output = weights[0] * mobilevit_logits + weights[1] * efficientnet_logits
        
        return output

# Load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hybrid_model = HybridGIModel(num_classes=8).to(device)
hybrid_model.load_state_dict(torch.load('hybrid_model_best.pth', map_location=device))
hybrid_model.eval()

efficientnet_model = create_efficientnet(num_classes=8)
efficientnet_model.load_weights("model_checkpoints/efficientnet_v2m_best_model.weights.h5")

print("✅ Models loaded successfully!")

## 6. Prepare Data

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

class KvasirDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
def load_dataset_paths(data_dir):
    image_paths, labels = [], []
    class_names = sorted(os.listdir(data_dir))
    class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}
    
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_dir, img_name))
                labels.append(class_to_idx[class_name])
    
    return image_paths, labels, class_names

image_paths, labels, class_names = load_dataset_paths(data_dir)

# Split
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.3, random_state=42, stratify=labels
)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

train_dataset = KvasirDataset(train_paths, train_labels, transform=transform)
val_dataset = KvasirDataset(val_paths, val_labels, transform=transform)
test_dataset = KvasirDataset(test_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"✅ Data ready: Train={len(train_paths)}, Val={len(val_paths)}, Test={len(test_paths)}")

## 7. Extract Features from Hybrid Model

In [ ]:
from tqdm import tqdm

def extract_features(model, efficientnet_model, data_loader, device):
    """Extract hybrid features for CatBoost"""
    features_list = []
    labels_list = []
    
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Extracting features"):
            images = images.to(device)
            
            # Get EfficientNet features
            images_np = images.cpu().numpy().transpose(0, 2, 3, 1)
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            images_np = images_np * std + mean
            images_np = np.clip(images_np * 255, 0, 255)
            
            eff_logits = efficientnet_model.predict(images_np, verbose=0)
            eff_logits = torch.from_numpy(eff_logits).float().to(device)
            
            # Get hybrid features (concatenated logits)
            hybrid_features = model(images, eff_logits, return_features=True)
            
            features_list.append(hybrid_features.cpu().numpy())
            labels_list.append(labels.numpy())
    
    features = np.vstack(features_list)
    labels = np.concatenate(labels_list)
    
    return features, labels

print("Extracting training features...")
X_train, y_train = extract_features(hybrid_model, efficientnet_model, train_loader, device)

print("Extracting validation features...")
X_val, y_val = extract_features(hybrid_model, efficientnet_model, val_loader, device)

print("Extracting test features...")
X_test, y_test = extract_features(hybrid_model, efficientnet_model, test_loader, device)

print(f"\n✅ Features extracted!")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

## 8. Optuna TPE Optimization for CatBoost

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

def objective(trial):
    """Optuna objective function"""
    
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'task_type': 'GPU',
        'verbose': False,
        'random_seed': 42
    }
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

print("🚀 Starting Optuna TPE optimization...")
print("This will run 50 trials to find the best CatBoost hyperparameters\n")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n✅ Optimization complete!")
print(f"\nBest validation accuracy: {study.best_value:.4f}")
print(f"\nBest parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

## 9. Train Final CatBoost Model

In [ ]:
# Train with best parameters
best_params = study.best_params
best_params['task_type'] = 'GPU'
best_params['verbose'] = True
best_params['random_seed'] = 42

print("Training final CatBoost model with best parameters...\n")

final_model = CatBoostClassifier(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50,
    plot=True
)

# Save model
final_model.save_model('catboost_optimized.cbm')
print("\n✅ Model saved as catboost_optimized.cbm")

## 10. Evaluate on Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Predict
y_pred_test = final_model.predict(X_test)

# Calculate metrics
test_accuracy = accuracy_score(y_test, y_pred_test)

print("="*60)
print("FINAL TEST RESULTS - OPTIMIZED MODEL")
print("="*60)
print(f"\n🎯 Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"\n📊 Improvement over base hybrid: {(test_accuracy - 0.8958)*100:.2f}%")

# Classification report
print("\n" + "="*60)
print("Classification Report:")
print("="*60)
print(classification_report(y_test, y_pred_test, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=class_names, yticklabels=class_names)
plt.title(f'Confusion Matrix - Optimized CatBoost\nTest Accuracy: {test_accuracy:.4f}', 
          fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('catboost_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Evaluation complete!")

## 11. Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = final_model.get_feature_importance()

# Plot
plt.figure(figsize=(12, 6))
plt.bar(range(len(feature_importance)), feature_importance)
plt.xlabel('Feature Index (0-7: MobileViT, 8-15: EfficientNet)', fontsize=12)
plt.ylabel('Importance', fontsize=12)
plt.title('CatBoost Feature Importance\n(Which model contributes more?)', fontsize=14, fontweight='bold')
plt.axvline(x=7.5, color='red', linestyle='--', label='MobileViT | EfficientNet')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

mobilevit_importance = feature_importance[:8].sum()
efficientnet_importance = feature_importance[8:].sum()

print("\n📊 Model Contribution:")
print(f"MobileViT features: {mobilevit_importance:.2f}")
print(f"EfficientNet features: {efficientnet_importance:.2f}")
print(f"\nDominant model: {'MobileViT' if mobilevit_importance > efficientnet_importance else 'EfficientNet'}")

## 12. Optuna Optimization History

In [ ]:
import pandas as pd

# Get trial history
df = study.trials_dataframe()

# Plot optimization history
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(df['number'], df['value'], marker='o', alpha=0.6)
plt.axhline(y=study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.4f}')
plt.xlabel('Trial', fontsize=12)
plt.ylabel('Validation Accuracy', fontsize=12)
plt.title('Optuna Optimization History', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(df['number'], df['value'].cummax(), marker='o', color='green', alpha=0.6)
plt.xlabel('Trial', fontsize=12)
plt.ylabel('Best Accuracy So Far', fontsize=12)
plt.title('Best Accuracy Progress', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('optuna_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Save results
df.to_csv('optuna_trials.csv', index=False)
print("\n✅ Optimization history saved to optuna_trials.csv")

## 13. Download Results

In [ ]:
from google.colab import files

# Create summary
with open('optimization_summary.txt', 'w') as f:
    f.write("="*60 + "\n")
    f.write("HYBRID MODEL OPTIMIZATION RESULTS\n")
    f.write("Optuna TPE + CatBoost\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"Base Hybrid Model Accuracy: 89.58%\n")
    f.write(f"Optimized CatBoost Accuracy: {test_accuracy*100:.2f}%\n")
    f.write(f"Improvement: {(test_accuracy - 0.8958)*100:.2f}%\n\n")
    
    f.write("Best Hyperparameters:\n")
    f.write("-" * 40 + "\n")
    for key, value in study.best_params.items():
        f.write(f"{key}: {value}\n")
    
    f.write("\n" + "="*60 + "\n")
    f.write("Feature Importance:\n")
    f.write("="*60 + "\n")
    f.write(f"MobileViT contribution: {mobilevit_importance:.2f}\n")
    f.write(f"EfficientNet contribution: {efficientnet_importance:.2f}\n")

print("📥 Downloading results...\n")

files_to_download = [
    'catboost_optimized.cbm',
    'catboost_confusion_matrix.png',
    'feature_importance.png',
    'optuna_history.png',
    'optuna_trials.csv',
    'optimization_summary.txt'
]

for filename in files_to_download:
    if os.path.exists(filename):
        files.download(filename)
        print(f"✅ Downloaded: {filename}")

print("\n🎉 All results downloaded!")

## 🎯 Summary

This notebook:
1. ✅ Loaded your trained hybrid model
2. ✅ Extracted features from both MobileViT and EfficientNet
3. ✅ Used Optuna TPE to optimize CatBoost hyperparameters (50 trials)
4. ✅ Trained optimized CatBoost meta-learner
5. ✅ Evaluated on test set
6. ✅ Analyzed feature importance
7. ✅ Downloaded all results

### Expected Results:
- **Base Hybrid**: 89.58%
- **Optimized CatBoost**: 91-93% (expected)

### Why This Works:
- CatBoost learns optimal combination of hybrid features
- Optuna finds best hyperparameters automatically
- Meta-learning captures non-linear relationships
- GPU acceleration makes it fast!